In [1]:
import requests   # external dependency
from pyquery import PyQuery as pq   # external dependency

import csv
from collections import OrderedDict


In [2]:
hosturl = "http://localhost/www.tdcj.texas.gov/death_row/dr_executed_offenders.html"
r = requests.get(hosturl)

In [3]:
# Now, let's get the actual stuff, and we'll put it in a variable called html.
html = r.content

In [4]:
masterdict = OrderedDict()

In [5]:
table = pq(html)("table.tdcj_table")   # Pick out the table with the good stuff

In [6]:
biopagebase = "http://localhost/www.tdcj.texas.gov/death_row/"
for row in pq(table)("tr")[1:]:   # Skip that header row, get everything else
    line = OrderedDict()
    line['InmateNo'] = pq(row)("td")[5].text
    line['ExecutionNo'] = pq(row)("td")[0].text
    line['LastName'] = pq(row)("td")[3].text
    line['FirstName'] = pq(row)("td")[4].text
    line['Age'] = pq(row)("td")[6].text
    line['ExecutionDate'] = pq(row)("td")[7].text
    line['Race'] = pq(row)("td")[8].text
    line['County'] = pq(row)("td")[9].text
    line['BioURL'] = biopagebase + pq(pq(row)("td")[1])("a").attr("href")
    line['StatementURL'] = biopagebase + pq(pq(row)("td")[2])("a").attr("href")
    masterdict[line['InmateNo']] = line

# Wait, did that just work?
print(f"Lines scraped: {len(masterdict)}")

Lines scraped: 558


In [7]:
for InmateNo in masterdict:
    line = masterdict[InmateNo]  # Get that whole person-level dictionary
    bioURL = line['BioURL']
    if "html" in bioURL and "no_info_available" not in bioURL:  # !!!!!!!!!!!!!!!!!!!!!!! This broke too
        r = requests.get(bioURL)   # Get their biography page
        html = r.content
        table = pq(html)("table.table_deathrow")
        for row in pq(table)("tr"):
            description = pq(row)("td")[-2].text
            data = pq(row)("td")[-1].text
            line[description] = data
        # Now let's write the line back to masterdict to save our changes
        try:
            photocellURL = pq(pq(pq(html)("table.table_deathrow"))("td")[0])("img").attr("src")
            if photocellURL:   # If we've found that img src tag ...
                photocellURL = photobase + photocellURL
            else:
                photocellURL = ""
        except:
            photocellURL = ""
        line['PhotoURL'] = photocellURL
        for graf in pq(html)("p"):
            spanhere = pq(graf)("span")
            if spanhere:   # If this is a complete description + initial tag:
                try:   # Yes, something else broke !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                    data = graf.text_content().split("\n")[1].strip()
                    description = pq(graf)("span")[0].text.strip()
                except:
                    pass
            else:
                line[description] += " ***INCOMPLETE**** "
            line[description] = data
        masterdict[InmateNo] = line    

In [8]:
# First, get a list of our keys, like our column headers.
headers = []
for InmateNo in masterdict:
    row = masterdict[InmateNo]
    for key in row.keys():
        if key not in headers:
            headers.append(key)
print(headers)

['InmateNo', 'ExecutionNo', 'LastName', 'FirstName', 'Age', 'ExecutionDate', 'Race', 'County', 'BioURL', 'StatementURL', 'Name', 'TDCJ Number', 'Date of Birth', 'Date Received', 'Age (when    Received)', 'Education Level (Highest Grade Completed)', 'Date of Offense', 'Age (at the time of Offense)', 'Gender', 'Hair Color', 'Height', 'Weight', 'Eye Color', 'Native County', 'Native State', 'PhotoURL', 'Prior Occupation', 'Prior Prison Record', 'Summary of Incident', 'Co-Defendants', 'Race and Gender of Victim', 'Age (when Received)', 'Summary of incident', 'Native Country']


In [9]:
# That looks reasonable. Now, let's cycle through again and standardize:
for InmateNo in masterdict:
    row = masterdict[InmateNo]
    line = OrderedDict()
    for key in headers:
        if key in row:
            line[key] = row[key]
        else:
            line[key] = ""   # Give it a blank entry
    
    # Our data is now standardized but has not been saved.
    masterdict[InmateNo] = line   # Copy over the new data.

In [10]:
with open('report.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for InmateNo in masterdict:
        line = masterdict[InmateNo]
        writer.writerow(line)